# 🧠 Quark Brain Simulation - Unified Kaggle Training Environment

This notebook provides a comprehensive, Kaggle-optimized environment for running various brain simulation tasks, migrating functionalities from existing cloud configurations (`skypilot`, `ray`, etc.) to leverage Kaggle's free GPU resources.

## Features:
- **Unified Interface**: A single notebook to run different types of brain simulations.
- **Multi-Framework Support**: Includes support for PyTorch, Brian2, and NEST for different simulation needs.
- **GPU Acceleration**: Optimized to use Kaggle's Tesla T4/P100 GPUs.
- **Cost-Effective**: Reduces reliance on paid cloud services for development and experimentation.
- **Interactive Visualization**: Real-time plotting of simulation metrics.
- **Modular Design**: Easily select and run specific simulation tasks.


In [ ]:
# --- 1. Setup and Environment Configuration ---
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
import json
import warnings
import time

# Suppress warnings
warnings.filterwarnings('ignore')

# Matplotlib setup
plt.style.use('seaborn-v0_8')
%matplotlib inline

# --- Installation of additional libraries ---
# Brian2 and NEST are not pre-installed on Kaggle
!pip install brian2 --quiet
!pip install nest-simulator --quiet

import brian2 as b2
import nest

print("="*60)
print("🧠 Quark Brain Simulation - Unified Kaggle Training Environment")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
print("="*60)

# --- Global Configuration ---
KAGGLE_WORKING_DIR = "/kaggle/working/"
SESSION_ID = f"unified_kaggle_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Simulation configuration
SIM_CONFIG = {
    "session_id": SESSION_ID,
    "environment": "kaggle_unified",
    "gpu_enabled": torch.cuda.is_available(),
    "simulation_type": "spiking_neural_network", # Options: "spiking_neural_network", "consciousness_model", "detailed_neuron"
    "duration_ms": 1000, # Simulation time in milliseconds
    "num_neurons": 1000,
    "log_interval": 100 # Log metrics every 100ms
}

# Save config
with open(os.path.join(KAGGLE_WORKING_DIR, 'sim_config.json'), 'w') as f:
    json.dump(SIM_CONFIG, f, indent=2)

print("\nConfiguration saved. Ready for simulation.")
print(json.dumps(SIM_CONFIG, indent=2))


---
## 2. Simulation Modules

This section contains different simulation modules that replicate the functionalities from the cloud configurations. You can choose which simulation to run.


In [ ]:
# --- Spiking Neural Network Simulation (like Brian2/NEST configs) ---
def run_spiking_network_simulation(config):
    """
    Simulates a large-scale spiking neural network, similar to what you'd run with Brian2 or NEST.
    """
    print("\n--- 🚀 Starting Spiking Neural Network (SNN) Simulation ---")
    start_time = time.time()
    
    # Brian2 implementation
    b2.start_scope()
    
    num_neurons = config['num_neurons']
    duration = config['duration_ms'] * b2.ms
    
    # Neuron model
    eqs = '''
    dv/dt = (I-v)/ (20*ms) : 1
    I : 1
    '''
    G = b2.NeuronGroup(num_neurons, eqs, threshold='v>1', reset='v=0', method='exact')
    G.I = '1.5 * rand()'
    
    # Synapses
    S = b2.Synapses(G, G, on_pre='v_post += 0.2')
    S.connect(p=0.1)
    
    # Monitors
    spike_monitor = b2.SpikeMonitor(G)
    pop_rate_monitor = b2.PopulationRateMonitor(G)
    
    # Run simulation
    b2.run(duration)
    
    end_time = time.time()
    print(f"✅ SNN Simulation finished in {end_time - start_time:.2f} seconds.")
    
    # --- Visualization ---
    plt.figure(figsize=(15, 6))
    
    # Raster plot
    plt.subplot(1, 2, 1)
    plt.plot(spike_monitor.t/b2.ms, spike_monitor.i, '.k', markersize=2)
    plt.title('Raster Plot of Neuron Spikes')
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron Index')
    
    # Population firing rate
    plt.subplot(1, 2, 2)
    plt.plot(pop_rate_monitor.t/b2.ms, pop_rate_monitor.rate/b2.Hz)
    plt.title('Population Firing Rate')
    plt.xlabel('Time (ms)')
    plt.ylabel('Firing Rate (Hz)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(KAGGLE_WORKING_DIR, 'snn_simulation_results.png'))
    plt.show()
    
    # --- Metrics ---
    metrics = {
        'simulation_type': 'SNN (Brian2)',
        'duration_s': (end_time - start_time),
        'total_spikes': len(spike_monitor.i),
        'mean_firing_rate_hz': np.mean(pop_rate_monitor.rate/b2.Hz)
    }
    
    return metrics


# --- Consciousness Model Training (like PyTorch/DeepSpeed configs) ---
def run_consciousness_model_training(config):
    """
    Trains a simplified consciousness model, demonstrating a PyTorch-based workflow.
    """
    print("\n--- 🎯 Starting Consciousness Model Training ---")
    start_time = time.time()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Simple model
    model = nn.Sequential(
        nn.Linear(config['num_neurons'], 512),
        nn.ReLU(),
        nn.Linear(512, 1),
        nn.Sigmoid()
    ).to(device)

    # Dummy data
    X = torch.randn(5000, config['num_neurons']).to(device)
    y = torch.randint(0, 2, (5000, 1), dtype=torch.float32).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    losses = []
    for epoch in range(10): # Short training demo
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if (epoch + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

    end_time = time.time()
    print(f"✅ Consciousness Model Training finished in {end_time - start_time:.2f} seconds.")

    # --- Visualization ---
    plt.figure(figsize=(8, 5))
    plt.plot(losses)
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(os.path.join(KAGGLE_WORKING_DIR, 'consciousness_model_training.png'))
    plt.show()

    # --- Metrics ---
    metrics = {
        'simulation_type': 'Consciousness Model (PyTorch)',
        'duration_s': (end_time - start_time),
        'final_loss': losses[-1]
    }
    return metrics

# --- Detailed Neuron Simulation (like NEURON/Arbor configs) ---
def run_detailed_neuron_simulation(config):
    """
    Simulates a single detailed neuron, demonstrating a NEST-based workflow.
    """
    print("\n--- 🔬 Starting Detailed Neuron Simulation (NEST) ---")
    start_time = time.time()

    nest.ResetKernel()
    
    # Create a neuron and a voltmeter
    neuron = nest.Create("iaf_psc_alpha")
    voltmeter = nest.Create("voltmeter")
    
    # Configure voltmeter to record membrane potential
    nest.SetStatus(voltmeter, {"withgid": True, "withtime": True})

    # Create a spike generator and connect it to the neuron
    spike_generator = nest.Create("spike_generator")
    nest.SetStatus(spike_generator, {"spike_times": [10.0, 20.0, 50.0, 80.0, 100.0]})
    nest.Connect(spike_generator, neuron)
    
    # Connect voltmeter to the neuron
    nest.Connect(voltmeter, neuron)
    
    # Run simulation
    nest.Simulate(config['duration_ms'])
    
    end_time = time.time()
    print(f"✅ Detailed Neuron Simulation finished in {end_time - start_time:.2f} seconds.")

    # --- Visualization ---
    events = nest.GetStatus(voltmeter, "events")[0]
    times = events["times"]
    voltages = events["V_m"]
    
    plt.figure(figsize=(8, 5))
    plt.plot(times, voltages)
    plt.title('Membrane Potential of a Single Neuron')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (mV)')
    plt.grid(True)
    plt.savefig(os.path.join(KAGGLE_WORKING_DIR, 'detailed_neuron_simulation.png'))
    plt.show()

    # --- Metrics ---
    metrics = {
        'simulation_type': 'Detailed Neuron (NEST)',
        'duration_s': (end_time - start_time),
        'num_spikes_recorded': len(times)
    }
    return metrics


---
## 3. Simulation Execution and Results

This section runs the selected simulation and logs the results.


In [ ]:
# --- Simulation Execution ---

# Choose the simulation to run based on SIM_CONFIG
simulation_type = SIM_CONFIG.get("simulation_type", "spiking_neural_network")

results = {}

if simulation_type == "spiking_neural_network":
    results = run_spiking_network_simulation(SIM_CONFIG)
elif simulation_type == "consciousness_model":
    results = run_consciousness_model_training(SIM_CONFIG)
elif simulation_type == "detailed_neuron":
    results = run_detailed_neuron_simulation(SIM_CONFIG)
else:
    print(f"Unknown simulation type: {simulation_type}")


# --- Final Report ---
print("\n" + "="*60)
print("📊 Simulation Report")
print("="*60)

final_report = {
    "session_id": SESSION_ID,
    "timestamp": datetime.now().isoformat(),
    "config": SIM_CONFIG,
    "results": results
}

print(json.dumps(final_report, indent=2))

# Save report
with open(os.path.join(KAGGLE_WORKING_DIR, 'final_report.json'), 'w') as f:
    json.dump(final_report, f, indent=2)

print(f"\n✅ Report saved to {KAGGLE_WORKING_DIR}final_report.json")
print("🎉 Unified Kaggle Training Notebook execution complete!")
